#### In this notebook file, we convert the dollar amounts of the damages due to hurricanes to the Consumer Price Index (CPI) adjusted to 2021 values. The formula we use is, 
###### USD in 2021 = (CPI in 2021 / CPI in the historic year ) * USD in the historic year

In [273]:
import pandas as pd
import numpy as np
import sqlalchemy
import os
import calendar
pd.set_option('display.precision', 2)
# from test import connection
import json

In [274]:
df = pd.read_json('static/js/storms.json')
df

,NameYear,AreasAffected,Damage,Deaths
0,Katrina 2005,"Bahamas, United States Gulf Coast",125000000000,1836
1,Maria 2017,"Lesser Antilles, Virgin Islands, Puerto Rico, ...",91600000000,3057
2,Irma 2017,"Cape Verde, The Caribbean, Virgin Islands, Cub...",64800000000,138
3,Andrew 1992,"The Bahamas, Florida, United States Gulf Coast",26500000000,65
4,Ivan 2004,"The Caribbean, Venezuela, United States Gulf C...",23300000000,124
...,...,...,...,...
56,Anna 1961,"Windward Islands, Colombia, Venezuela, Central...",300,1
57,Florence 2006,"Bermuda, Newfoundland, United States East Coas...",200,0
58,Doria 1967,United States East Coast,150,3
59,Erika 2003,"Florida, Mexico, Southern Texas",100,2


In [275]:
df2 = pd.read_csv('Data/USCPI_1900-2022.csv')
df2


,Year,U.S. Consumer Price Index *
0,1900,8.14
1,1901,8.24
2,1902,8.34
3,1903,8.53
4,1904,8.63
...,...,...
117,2017,245.12
118,2018,251.11
119,2019,255.66
120,2020,258.81


In [276]:
df2['c'] =  df2.loc[df2['Year'] == 2021, 'U.S. Consumer Price Index *'] 

df2 = df2.fillna(0)

df2

,Year,U.S. Consumer Price Index *,c
0,1900,8.14,0.00
1,1901,8.24,0.00
2,1902,8.34,0.00
3,1903,8.53,0.00
4,1904,8.63,0.00
...,...,...,...
117,2017,245.12,0.00
118,2018,251.11,0.00
119,2019,255.66,0.00
120,2020,258.81,0.00


In [277]:
df3 = df2.tail(1)
var =  df3.values[0]
current_cpi = var[2]
print(current_cpi)

270.97


In [278]:
df4 = df2.replace(to_replace = 0.00, value = current_cpi)
df4

,Year,U.S. Consumer Price Index *,c
0,1900,8.14,270.97
1,1901,8.24,270.97
2,1902,8.34,270.97
3,1903,8.53,270.97
4,1904,8.63,270.97
...,...,...,...
117,2017,245.12,270.97
118,2018,251.11,270.97
119,2019,255.66,270.97
120,2020,258.81,270.97


In [279]:
df4['relative_index'] = df4['c']/df4['U.S. Consumer Price Index *']

df4 = df4.drop(['U.S. Consumer Price Index *','c'], axis=1)
df4.dtypes


Year                int64
relative_index    float64
dtype: object

In [280]:
df['Year'] = df['NameYear'].str.extract('.*(\d{4})', expand = False)
df

df = df.astype({"Year":"int"})
df.dtypes

NameYear         object
AreasAffected    object
Damage            int64
Deaths            int64
Year              int32
dtype: object

In [281]:
df5 = pd.merge(df, df4, on=['Year','Year'], how="inner")
df5 = df5.rename(columns={"Damage": "Damage_not_adjusted"})
df5

,NameYear,AreasAffected,Damage_not_adjusted,Deaths,Year,relative_index
0,Katrina 2005,"Bahamas, United States Gulf Coast",125000000000,1836,2005,1.39
1,Rita 2005,"Cuba, United States Gulf Coast",12000000000,125,2005,1.39
2,Stan 2005,"Costa Rica, Nicaragua, Honduras, El Salvador, ...",3960000000,1668,2005,1.39
3,Cindy 2005,"Yucatin Peninsula, The Carolinas, Alabama, Mis...",320000000,3,2005,1.39
4,Ophelia 2005,"The Bahamas, Eastern Coast of the United State...",70000000,3,2005,1.39
...,...,...,...,...,...,...
56,Klaus 1990,"Lesser Antilles, Puerto Rico, Hispaniola, Turk...",1000000,11,1990,2.07
57,Floyd 1987,"Cuba, Florida, The Bahamas",500,1,1987,2.39
58,Arlene 1987,"The Bahamas, Bermuda",8,0,1987,2.39
59,Anna 1961,"Windward Islands, Colombia, Venezuela, Central...",300,1,1961,9.06


In [282]:
df5['Damage'] = df5['Damage_not_adjusted'] * df5['relative_index']
df5['Damage'] = df5['Damage'] / 1000000000

df5 = df5.drop(['Damage_not_adjusted','relative_index','Year'], axis=1)
df5

,NameYear,AreasAffected,Deaths,Damage
0,Katrina 2005,"Bahamas, United States Gulf Coast",1836,1.73e+02
1,Rita 2005,"Cuba, United States Gulf Coast",125,1.66e+01
2,Stan 2005,"Costa Rica, Nicaragua, Honduras, El Salvador, ...",1668,5.49e+00
3,Cindy 2005,"Yucatin Peninsula, The Carolinas, Alabama, Mis...",3,4.44e-01
4,Ophelia 2005,"The Bahamas, Eastern Coast of the United State...",3,9.71e-02
...,...,...,...,...
56,Klaus 1990,"Lesser Antilles, Puerto Rico, Hispaniola, Turk...",11,2.07e-03
57,Floyd 1987,"Cuba, Florida, The Bahamas",1,1.19e-06
58,Arlene 1987,"The Bahamas, Bermuda",0,1.91e-08
59,Anna 1961,"Windward Islands, Colombia, Venezuela, Central...",1,2.72e-06


In [283]:
df5 = df5.loc[:,['NameYear','AreasAffected','Damage','Deaths']]
df5 = df5.set_index('NameYear')
df5

,AreasAffected,Damage,Deaths
NameYear,,,
Katrina 2005,"Bahamas, United States Gulf Coast",1.73e+02,1836
Rita 2005,"Cuba, United States Gulf Coast",1.66e+01,125
Stan 2005,"Costa Rica, Nicaragua, Honduras, El Salvador, ...",5.49e+00,1668
Cindy 2005,"Yucatin Peninsula, The Carolinas, Alabama, Mis...",4.44e-01,3
Ophelia 2005,"The Bahamas, Eastern Coast of the United State...",9.71e-02,3
...,...,...,...
Klaus 1990,"Lesser Antilles, Puerto Rico, Hispaniola, Turk...",2.07e-03,11
Floyd 1987,"Cuba, Florida, The Bahamas",1.19e-06,1
Arlene 1987,"The Bahamas, Bermuda",1.91e-08,0


In [286]:
result = df5.to_json(orient="index")
parsed = json.loads(result)
result = ('static/js/adjusted.json')  